<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

### How can I create technical chart analysis of real time price feeds?

In this notebook, we will be creating a simple technical analysis chart of real time price feeds. It will include rates of change for prices overlayed by housing demand metrics via home purchases.

#### What will you create in this notebook?

<p align="center">
  <img src="../../images/technical_pf_usa_vs_demand_chart.png" alt="Alt text">
</p>

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/price_feed_trader_resources/trader_charts.ipynb)

### 1. Import the Parcl Labs Python Library

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

In [ ]:
import os
import numpy as np
import pandas as pd
from parcllabs import ParclLabsClient
from parcllabs.beta.ts_stats import TimeSeriesAnalysis
from parcllabs.beta.charting.name_formats import format_names, create_ticker
from parcllabs.beta.charting.technical_charts import build_technical_chart

client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for Price Feed Markets on Parcl Exchange

In [ ]:
sales_pricefeed_markets = client.search.markets.retrieve(
    query='United States',
    sort_by='PARCL_EXCHANGE_MARKET',
    sort_order='DESC',
    limit=1
)

sales_pricefeed_markets['clean_name'] = sales_pricefeed_markets['name'].apply(format_names)
sales_pricefeed_markets['ticker'] = sales_pricefeed_markets['clean_name'].apply(create_ticker)

In [ ]:
sales_pricefeed_markets.head()

### 3. Load the data for sales price feeds and sales demand

In [ ]:
# lets retrieve data back to 2011 for these price feeds
START_DATE = '2019-01-01'
sales_price_feeds = client.price_feed.price_feed.retrieve(
    parcl_ids=sales_pricefeed_markets['parcl_id'].tolist(),
    start_date=START_DATE,
    limit=1000,  # expand the limit to 1000, these are daily series
    auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)

In [ ]:
# get supply and demand
demand = client.market_metrics.housing_event_counts.retrieve(
    parcl_ids=sales_pricefeed_markets['parcl_id'].tolist(),
    start_date=START_DATE,
    limit=1000,  # expand the limit to 1000, these are daily series
)

### Prepare the data for analysis/charting

In [ ]:
# lets analyze the US housing market on the parcl exchange with parcl_id: 5826765
# we can confirm via:
pid = 5826765
sales_pricefeed_markets[sales_pricefeed_markets['parcl_id'] == pid]

In [ ]:
pricefeed = sales_price_feeds[sales_price_feeds['parcl_id'] == pid].copy()
demand = demand[demand['parcl_id'] == pid].copy()
# sort ascending order
pricefeed = pricefeed.sort_values('date')
demand = demand.sort_values('date')
metadata = sales_pricefeed_markets[sales_pricefeed_markets['parcl_id'] == pid]
ticker = metadata['ticker'].values[0]
name = metadata['name'].values[0]

pf_ts_analysis = TimeSeriesAnalysis(pricefeed, 'date', 'price_feed', freq='D')
pf_rate_of_change_stats = pf_ts_analysis.calculate_changes()

demand_ts_analysis = TimeSeriesAnalysis(demand, 'date', 'sales', freq='M')
demand_rate_of_change_stats = demand_ts_analysis.calculate_changes()

In [ ]:

# get last information for pricefeed
last_pricefeed_value = pf_rate_of_change_stats['last']['value']
last_pricefeed_date_str = pf_rate_of_change_stats['last']['date']
last_pricefeed_date = pd.to_datetime(last_pricefeed_date_str).strftime('%-d %B')
last_pf_date = pd.to_datetime(last_pricefeed_date_str).strftime('%-d-%B-%Y')

# get last information for demand
last_demand_value = demand_rate_of_change_stats['last']['value']
last_sales_month = pd.to_datetime(demand_rate_of_change_stats['last']['date']).strftime('%b')

In [ ]:
# calculate daily return
pricefeed['daily_return'] = pricefeed['price_feed'].pct_change()
# drop na values
pricefeed = pricefeed.dropna().reset_index(drop=True)
pricefeed['month'] = pricefeed['date'].dt.month
demand['month'] = demand['date'].dt.month
# start and end of the month change
monthly_returns = pricefeed.groupby('month')['price_feed'].agg(lambda x: x.iloc[-1] / x.iloc[0] - 1).reset_index(name='volColor')

In [ ]:
# merge monthly returns
pricefeed = pricefeed.merge(monthly_returns, on='month', how='left')
demand = demand.merge(monthly_returns, on='month', how='left')

In [ ]:
# Add volColor column directly to data_secondary
demand['volColor'] = np.where(demand['volColor'] >= 0, 'green', 'red')

In [ ]:
# chart labels
msg = f"""
<b>52w High</b> {pf_rate_of_change_stats['52_week_high']['value']:.2f} 
<b>52w Low</b> {pf_rate_of_change_stats['52_week_low']['value']:.2f} 
<b>Last</b> {pf_rate_of_change_stats['last']['value']:.2f} 
<b>YoY Change</b> {pf_rate_of_change_stats['12_month']['value_diff']} ({pf_rate_of_change_stats['12_month']['percent_change']*100:.02f}%)
"""
msg = msg.replace('\n', '')


ticker_msg = f"<b>{ticker}</b> {name} Parcl Exchange"
volume_msg = f'Sales Volume (Monthly) {last_demand_value:,} ({last_sales_month})'
pricefeed_msg = f'{ticker} (Daily) ${last_pricefeed_value} ({last_pricefeed_date})'

In [ ]:
build_technical_chart(
    line_chart_df=pricefeed,
    bar_chart_df=demand,
    value_name_main='price_feed',
    value_name_secondary='sales',
    header_left_primary_text=ticker_msg,
    header_left_secondary_text=last_pf_date,
    header_right_primary_text=msg,
    line_chart_left_primary_text=pricefeed_msg,
    bar_chart_left_primary_text=volume_msg,
    # save_path='chart.png',
)